In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
import traceback

import torch

from sem2SQL import transform
from src import args as arg
from src import utils
from src.models.model import IRNet
from src.rule import semQL

import subprocess

from dummy_input import DUMMY_INPUT
from src.rule.sem_utils import alter_column0, alter_inter, alter_not_in
from src.utils import load_data_new

MODEL_PATH = "./saved_model/IRNet_pretrained.model"
EMBEDS_PATH = "./data/glove.42B.300d.txt"
QUESTION_DATA_PATH = "./data/custom/question.json"
TABLE_DATA_PATH = "./data/custom/tables.json"
PREDICT_LF_PATH = "./data/custom/predict_lf.json"

CUDA = False

In [3]:
arg_parser = arg.init_arg_parser()
args = arg_parser.parse_args("--dataset ./data/custom --glove_embed_path ./data/glove.42B.300d.txt --cuda --epoch 50 --loss_epoch_threshold 50 --sketch_loss_coefficie 1.0 --beam_size 5 --seed 90 --save ${save_name} --embed_size 300 --sentence_features --column_pointer --hidden_size 300 --lr_scheduler --lr_scheduler_gammar 0.5 --att_vec_size 300 --load_model ./saved_model/IRNet_pretrained.model".split())

In [4]:
grammar = semQL.Grammar()
model = IRNet(args, grammar)

if CUDA: model.cuda()

print('load pretrained model from %s' % MODEL_PATH)
pretrained_model = torch.load(MODEL_PATH,
                              map_location=lambda storage, loc: storage)
import copy
pretrained_modeled = copy.deepcopy(pretrained_model)
for k in pretrained_model.keys():
    if k not in model.state_dict().keys():
        del pretrained_modeled[k]

model.load_state_dict(pretrained_modeled)

model.word_emb = utils.load_word_emb(EMBEDS_PATH)

model.eval()

Use Column Pointer:  True
load pretrained model from ./saved_model/IRNet_pretrained.model
Loading word embedding from ./data/glove.42B.300d.txt


IRNet(
  (encoder_lstm): LSTM(300, 150, batch_first=True, bidirectional=True)
  (lf_decoder_lstm): LSTMCell(556, 300)
  (sketch_decoder_lstm): LSTMCell(556, 300)
  (decoder_cell_init): Linear(in_features=300, out_features=300, bias=True)
  (att_sketch_linear): Linear(in_features=300, out_features=300, bias=False)
  (att_lf_linear): Linear(in_features=300, out_features=300, bias=False)
  (sketch_att_vec_linear): Linear(in_features=600, out_features=300, bias=False)
  (lf_att_vec_linear): Linear(in_features=600, out_features=300, bias=False)
  (prob_att): Linear(in_features=300, out_features=1, bias=True)
  (prob_len): Linear(in_features=1, out_features=1, bias=True)
  (col_type): Linear(in_features=4, out_features=300, bias=True)
  (sketch_encoder): LSTM(128, 64, batch_first=True, bidirectional=True)
  (production_embed): Embedding(46, 128)
  (type_embed): Embedding(10, 128)
  (att_project): Linear(in_features=428, out_features=300, bias=True)
  (N_embed): Embedding(5, 128)
  (query_vec

In [13]:
question_str = "How many customers do we have?"

In [14]:
DUMMY_INPUT["question"] = question_str
DUMMY_INPUT["question_toks"] = question_str.split(" ")

import json
with open(QUESTION_DATA_PATH, 'w', encoding='utf-8') as f:
    json.dump([DUMMY_INPUT], f, ensure_ascii=False, indent=4)

subprocess.run(["cd preprocess && bash run_me.sh ../data/custom/question.json ../data/custom/tables.json"], shell=True)

with open(TABLE_DATA_PATH) as inf:
    print("Loading data from %s" % TABLE_DATA_PATH)
    table_data = json.load(inf)

Loading data from ./data/custom/tables.json


In [20]:
question_data, table_data_new = load_data_new(QUESTION_DATA_PATH, table_data, use_small=False)

Loading data from ./data/custom/question.json


In [21]:
question_data

[{'db_id': 'customer',
  'query': 'SELECT count(*) FROM customer',
  'query_toks': ['SELECT', 'count', '(', '*', ')', 'FROM', 'customer'],
  'query_toks_no_value': ['select',
   'count',
   '(',
   '*',
   ')',
   'from',
   'customer'],
  'question': 'How many customers do we have?',
  'question_toks': ['how', 'many', 'customer', 'do', 'we', 'have?'],
  'sql': {'except': None,
   'from': {'conds': [], 'table_units': [['table_unit', 0]]},
   'groupby': [],
   'having': [],
   'intersect': None,
   'limit': None,
   'orderby': [],
   'select': [False, [[3, [0, [0, 0, False], None]]]],
   'union': None,
   'where': []},
  'names': ['*',
   'customer_Id',
   'name',
   'address',
   'birthday',
   'phone',
   'city'],
  'table_names': ['customer'],
  'col_set': ['*',
   'customer_Id',
   'name',
   'address',
   'birthday',
   'phone',
   'city'],
  'col_table': [-1, 0, 0, 0, 0, 0, 0],
  'keys': {'1': 1},
  'origin_question_toks': ['How', 'many', 'customers', 'do', 'we', 'have?'],
  'ques

In [17]:
table_data_new

{'customer': {'column_names': [[-1, '*'],
   [0, 'customer_Id'],
   [0, 'name'],
   [0, 'address'],
   [0, 'birthday'],
   [0, 'phone'],
   [0, 'city']],
  'column_names_original': [[-1, '*'],
   [0, 'customer_ID'],
   [0, 'name'],
   [0, 'address'],
   [0, 'birthday'],
   [0, 'phone'],
   [0, 'city']],
  'column_types': ['text', 'number', 'text', 'text', 'text', 'number', 'text'],
  'db_id': 'customer',
  'foreign_keys': [],
  'primary_keys': [1],
  'table_names': ['customer'],
  'table_names_original': ['customer']}}

In [22]:
json_datas = utils.get_json_data(model, table_data_new, question_data, beam_size=args.beam_size)

AssertionError: 
Found no NVIDIA driver on your system. Please check that you
have an NVIDIA GPU and installed a driver from
http://www.nvidia.com/Download/index.aspx

In [115]:
data_model = [
{
            "key": "o6WsUA0TqUqBDkuVqplMGg",
            "name": "Company",
            "prettyName": "Company",
            "isHidden": False,
            "isStatic": False,
            "isClient": False,
            "isPublic": True,
            "isPublicReadOnly": True,
            "hasProcessEvents": False,
            "description": None,
            "label": "CompanyName",
            "labelPlural": "Companies",
            "refKey": "pO_xm686M02sL3svyaZ6dA",
            "refOmlKey": "GCHbAzwWJ0eLsSv4AqjokA",
            "lastModifiedDate": None,
            "attributes": [
            {
                "key": "GUbYmeqPVUylkqAv8JSR5A",
                "name": "Id",
                "type": "Integer",
                "autoNumber": "Yes",
                "isPrimary": True,
                "isMandatory": True,
                "maxLength": None,
                "defaultValue": None,
                "fkRefKey": None
            },
            {
                "key": "_tJLsPg0LkmYY5jnarM3_A",
                "name": "CompanyName",
                "type": "Text",
                "autoNumber": "No",
                "isPrimary": False,
                "isMandatory": True,
                "maxLength": "50",
                "defaultValue": None,
                "fkRefKey": None
            },
            {
                "key": "5TBSW3drxUqp2VwzTl1CEQ",
                "name": "CompanyType",
                "type": "text",
                "autoNumber": "No",
                "isPrimary": False,
                "isMandatory": False,
                "maxLength": None,
                "defaultValue": None,
                "fkRefKey": None
            },
            {
                "key": "v2DGqYEfGk2mg_G3TxoxQQ",
                "name": "Phone",
                "type": "Phone Number",
                "autoNumber": "No",
                "isPrimary": False,
                "isMandatory": False,
                "maxLength": "20",
                "defaultValue": None,
                "fkRefKey": None
            },
            {
                "key": "2f_jBwJ9SUGYpx6dRWhwkw",
                "name": "Email",
                "type": "Text",
                "autoNumber": "No",
                "isPrimary": False,
                "isMandatory": False,
                "maxLength": "50",
                "defaultValue": None,
                "fkRefKey": None
            },
            {
                "key": "NZo+RPPmeE2f3Y9Xlds+6Q",
                "name": "Fax",
                "type": "Phone Number",
                "autoNumber": "No",
                "isPrimary": False,
                "isMandatory": False,
                "maxLength": "20",
                "defaultValue": None,
                "fkRefKey": None
            },
            {
                "key": "YLHmXL9JdEmLMz1Hjjjy6g",
                "name": "Website",
                "type": "Text",
                "autoNumber": "No",
                "isPrimary": False,
                "isMandatory": False,
                "maxLength": "50",
                "defaultValue": None,
                "fkRefKey": None
            },
            {
                "key": "_fXrXNXftEu7P3W2wqTRdw",
                "name": "Address",
                "type": "Text",
                "autoNumber": "No",
                "isPrimary": False,
                "isMandatory": False,
                "maxLength": "50",
                "defaultValue": None,
                "fkRefKey": None
            },
            {
                "key": "FOOSKFUTjEaoY7Aa0DqLjQ",
                "name": "PostalCode",
                "type": "Text",
                "autoNumber": "No",
                "isPrimary": False,
                "isMandatory": False,
                "maxLength": "50",
                "defaultValue": None,
                "fkRefKey": None
            },
            {
                "key": "sODcQszkFEGWPC4Do3sg5w",
                "name": "City",
                "type": "Text",
                "autoNumber": "No",
                "isPrimary": False,
                "isMandatory": False,
                "maxLength": "50",
                "defaultValue": None,
                "fkRefKey": None
            },
            {
                "key": "aYxCZxVXsEaihdpIHfZ7oQ",
                "name": "CountryId",
                "type": "Country Identifier",
                "autoNumber": "No",
                "isPrimary": False,
                "isMandatory": False,
                "maxLength": None,
                "defaultValue": None,
                "fkRefKey": "g53tlwX9bUmADkZysMq1MQ"
            }
            ],
            "staticRecords": None
        },
        {
            "key": "tblt_jcwLEqv8lfdDffdhg",
            "name": "Contact",
            "prettyName": "Contact",
            "isHidden": False,
            "isStatic": False,
            "isClient": False,
            "isPublic": True,
            "isPublicReadOnly": True,
            "hasProcessEvents": False,
            "description": None,
            "label": "Name",
            "labelPlural": "Contacts",
            "refKey": "oxPsCCtWqUiHxkcyFKiasw",
            "refOmlKey": "GCHbAzwWJ0eLsSv4AqjokA",
            "lastModifiedDate": None,
            "attributes": [
            {
                "key": "kh5eCrMfWkK0q3MFM1yfPg",
                "name": "Id",
                "type": "Integer",
                "autoNumber": "Yes",
                "isPrimary": True,
                "isMandatory": True,
                "maxLength": None,
                "defaultValue": None,
                "fkRefKey": None
            },
            {
                "key": "T01UV3fTXUWzDgOLMbi_Ag",
                "name": "Name",
                "type": "Text",
                "autoNumber": "No",
                "isPrimary": False,
                "isMandatory": True,
                "maxLength": "100",
                "defaultValue": None,
                "fkRefKey": None
            },
            {
                "key": "XiCWsEAOb0+tzDDhny4FIw",
                "name": "JobTitle",
                "type": "Text",
                "autoNumber": "No",
                "isPrimary": False,
                "isMandatory": False,
                "maxLength": "50",
                "defaultValue": None,
                "fkRefKey": None
            },
            {
                "key": "sYjDvc5yPEWi1zWO_AjYMw",
                "name": "Phone",
                "type": "Phone Number",
                "autoNumber": "No",
                "isPrimary": False,
                "isMandatory": False,
                "maxLength": "20",
                "defaultValue": None,
                "fkRefKey": None
            },
            {
                "key": "yApqpd8rgEq3usN9tkQ3Ww",
                "name": "Mobile",
                "type": "Phone Number",
                "autoNumber": "No",
                "isPrimary": False,
                "isMandatory": False,
                "maxLength": "20",
                "defaultValue": None,
                "fkRefKey": None
            },
            {
                "key": "CozvPOGBh0eP2Ny4Sxi3RQ",
                "name": "Email",
                "type": "Text",
                "autoNumber": "No",
                "isPrimary": False,
                "isMandatory": False,
                "maxLength": "250",
                "defaultValue": None,
                "fkRefKey": None
            },
            {
                "key": "qyMa_n_grk6AVC7t0cWByg",
                "name": "CompanyId",
                "type": "Company Identifier",
                "autoNumber": "No",
                "isPrimary": False,
                "isMandatory": False,
                "maxLength": None,
                "defaultValue": None,
                "fkRefKey": "o6WsUA0TqUqBDkuVqplMGg"
            },
            {
                "key": "o2HymHeTNEqT1pa23vXiRg",
                "name": "SecondaryEmail",
                "type": "Text",
                "autoNumber": "No",
                "isPrimary": False,
                "isMandatory": False,
                "maxLength": "250",
                "defaultValue": None,
                "fkRefKey": None
            }
            ],
            "staticRecords": None
        },
]

In [4]:
import json

In [2]:
import pandas as pd

In [116]:
new_data_model = []
for idx, ent in enumerate(data_model):
    for attr in ent['attributes']:
        attr['entIdx'] = idx
        attr['entKey'] = ent['key']
        attr['entName'] = ent['name']
        attr['entIsHidden'] = ent['isHidden']
        attr['entIsStatic'] = ent['isStatic']
        attr['entRefKey'] = ent['refKey']
        new_data_model.append(attr)

In [117]:
df = pd.DataFrame(new_data_model)

In [88]:
df.head()

,key,name,type,autoNumber,isPrimary,isMandatory,maxLength,defaultValue,fkRefKey,entKey,entName,entIsHidden,entIsStatic,entRefKey
0,GUbYmeqPVUylkqAv8JSR5A,Id,Integer,Yes,True,True,None,None,None,o6WsUA0TqUqBDkuVqplMGg,Company,False,False,pO_xm686M02sL3svyaZ6dA
1,_tJLsPg0LkmYY5jnarM3_A,CompanyName,Text,No,False,True,50,None,None,o6WsUA0TqUqBDkuVqplMGg,Company,False,False,pO_xm686M02sL3svyaZ6dA
2,5TBSW3drxUqp2VwzTl1CEQ,CompanyType,text,No,False,False,None,None,None,o6WsUA0TqUqBDkuVqplMGg,Company,False,False,pO_xm686M02sL3svyaZ6dA
3,v2DGqYEfGk2mg_G3TxoxQQ,Phone,Phone Number,No,False,False,20,None,None,o6WsUA0TqUqBDkuVqplMGg,Company,False,False,pO_xm686M02sL3svyaZ6dA
4,2f_jBwJ9SUGYpx6dRWhwkw,Email,Text,No,False,False,50,None,None,o6WsUA0TqUqBDkuVqplMGg,Company,False,False,pO_xm686M02sL3svyaZ6dA


In [122]:
def normalize_word(word):

    if len(word)<2:
        return [word.lower()]

    tokens = []
    b = 0 if (word[0].isalpha()) else 1
    i=1

    for char in word[1:]:

        # If the word has a lower case letter followed by an upper case letter then it's separated
        #  in two subtokens, with the uppercase letter being the cutting point
        # For example, createdBy would be divided in created and by
        # Note that the uppercase letter of the second word is replaced by its correspondent lower case letter
        if char.isupper() and word[i-1].islower() and (i-b)>1:
            tokens.append(word[b:i].lower())
            b = i

        # If the word has a sequence of upper case letters followed by a lower case letter then it's separated
        #  in two subtokens, with the last upper letter being the cutting point
        # For example, FRAPScase would be divided in frap and scase
        # Note that the uppercase letters are replaced by its correspondent lower case letters
        elif char.islower() and word[i-1].isupper() and (i-b)>1:
            if (i-b>2):
                tokens.append(word[b:i-1].lower())
            b = i-1

        # If the word has an alpha numeric character followed by a non alpha numeric character then it's separated
        #  in two subtokens, with the non alphabetic character being the cutting point
        # For example, created_By would be divided in created and by
        # Note that the uppercase letter of the second word is replaced by its correspondent lower case letter
        # Note2: the word has to have at least 2 characters to be considered valid
        elif (not char.isalpha()):
            if (word[i-1].isalpha()) and (i-b)>1:
                tokens.append(word[b:i].lower())
            b = i+1

        i+=1

    # Add last sub token
    # Note that the uppercase letter of the second word is replaced by its correspondent lower case letter
    # Note2: the word has to have at least 2 characters to be considered valid
    if b<len(word) and (i-b)>1:
        tokens.append(word[b:i].lower())

    return " ".join(tokens)

In [123]:
map_types = dict({'Text': 'text',
                  'Integer': 'number',
                  'Decimal': 'number',
                  'Long Integer': 'number',
                  'Boolean': 'number',
                  'Date time': 'time',
                  'Time': 'time',
                  'Date': 'time',
                  'Phone Number': 'number',
                  'Email': 'text',
                  'Currency': 'number',
                  'Categorical': 'text',
                  'Identifier': 'number'})

df['index'] = df.index + 1

all_column = [-1, "*"]

result = {
    'column_names': [all_column, *df.apply(lambda x: [x['entIdx'], normalize_word(x['name'])], axis=1).tolist()],
    'column_names_original': [all_column, *df.apply(lambda x: [x['entIdx'], x['name']], axis=1).tolist()],
    'column_types': df['type'].map(lambda x: 'number' if x.endswith('Identifier') else map_types.get(x, 'text')).tolist(),
    'db_id': '_'.join(df['entName'].unique().tolist()),
    'primary_keys': df.loc[df['isPrimary']==True]['index'].tolist(), 
    'table_names': df['entName'].unique().tolist(),
    'table_names_original': [normalize_word(x) for x in df['entName'].unique().tolist()]
}

fks = []
for fkref, idx in df.loc[~df['fkRefKey'].isnull()][['fkRefKey', 'index']].values.tolist():
    origin_key_idx = df.loc[(df['entKey'] == fkref) & (df['isPrimary'] == True)]['index'].tolist()
    if len(origin_key_idx) == 1:
        fks.append([idx, origin_key_idx[0]])
        
result['foreign_keys'] = fks


print(json.dumps(result, indent=4))

{
    "column_names": [
        [
            -1,
            "*"
        ],
        [
            0,
            "id"
        ],
        [
            0,
            "company name"
        ],
        [
            0,
            "company type"
        ],
        [
            0,
            "phone"
        ],
        [
            0,
            "email"
        ],
        [
            0,
            "fax"
        ],
        [
            0,
            "website"
        ],
        [
            0,
            "address"
        ],
        [
            0,
            "postal code"
        ],
        [
            0,
            "city"
        ],
        [
            0,
            "country id"
        ],
        [
            1,
            "id"
        ],
        [
            1,
            "name"
        ],
        [
            1,
            "job title"
        ],
        [
            1,
            "phone"
        ],
        [
            1,
            "mobile"
        ],
   

In [31]:
[attr['isPrimary'] for ent in df['attributes'].values.tolist() for attr in ent]

[True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

In [4]:
predicted = {
        "db_id": "customer",
        "query": "SELECT count(*) FROM customer",
        "query_toks": [
            "SELECT",
            "count",
            "(",
            "*",
            ")",
            "FROM",
            "customer"
        ],
        "query_toks_no_value": [
            "select",
            "count",
            "(",
            "*",
            ")",
            "from",
            "customer"
        ],
        "question": "How many customers do we have?",
        "question_toks": [
            "How",
            "many",
            "customers",
            "do",
            "we",
            "have",
            "?"
        ],
        "sql": {
            "except": None,
            "from": {
                "conds": [],
                "table_units": [
                    [
                        "table_unit",
                        0
                    ]
                ]
            },
            "groupBy": [],
            "having": [],
            "intersect": None,
            "limit": None,
            "orderBy": [],
            "select": [
                False,
                [
                    [
                        3,
                        [
                            0,
                            [
                                0,
                                0,
                                False
                            ],
                            None
                        ]
                    ]
                ]
            ],
            "union": None,
            "where": []
        }
    }

In [5]:
table = {
        "column_names": [
            [
                -1,
                "*"
            ],
            [
                0,
                "customer_Id"
            ],
            [
                0,
                "name"
            ],
            [
                0,
                "address"
            ],
            [
                0,
                "birthday"
            ],
            [
                0,
                "phone"
            ],
            [
                0,
                "city"
            ]
        ],
        "column_names_original": [
            [
                -1,
                "*"
            ],
            [
                0,
                "customer_ID"
            ],
            [
                0,
                "name"
            ],
            [
                0,
                "address"
            ],
            [
                0,
                "birthday"
            ],
            [
                0,
                "phone"
            ],
            [
                0,
                "city"
            ]
        ],
        "column_types": [
            "text",
            "number",
            "text",
            "text",
            "text",
            "number",
            "text"
        ],
        "db_id": "customer",
        "foreign_keys": [ ],
        "primary_keys": [
            1
        ],
        "table_names": [
            "customer"
        ],
        "table_names_original": [
            "customer"
        ]
    }

In [10]:
def _get_out_attributes(predicted_lf, table_data):
    out_attrs = []
    for agg_and_value in predicted_lf['sql']['select'][1]:
        agg_id = agg_and_value[0]
        if agg_id != 0:
            import ipdb; ipdb.set_trace()
            out_attrs.append(None)
            continue
        value = agg_and_value[1]
        unit_op = value[0]
        if unit_op != 0:
            import ipdb; ipdb.set_trace()
            out_attrs.append(None)
            continue
        for col in (value[1], value[2]):
            if not col:
                continue
            agg_id = col[0]
            if agg_id != 0:
                import ipdb; ipdb.set_trace()
                out_attrs.append(None)
                break
            cold_id = col[1]
            out_attrs.append(table_data['column_keys'][cold_id][1])
    return out_attrs

In [ ]:
_get_out_attributes(predicted, table)

> <ipython-input-10-2f165e409476>(7)_get_out_attributes()
      6             import ipdb; ipdb.set_trace()
----> 7             out_attrs.append(None)
      8             continue

ipdb> l
      2     out_attrs = []
      3     for agg_and_value in predicted_lf['sql']['select'][1]:
      4         agg_id = agg_and_value[0]
      5         if agg_id != 0:
      6             import ipdb; ipdb.set_trace()
----> 7             out_attrs.append(None)
      8             continue
      9         value = agg_and_value[1]
     10         unit_op = value[0]
     11         if unit_op != 0:
     12             import ipdb; ipdb.set_trace()

ipdb> agg_id
3
